In [1]:
import pandas as pd
import numpy as np

from collections import Counter
from itertools import permutations

import networkx as nx

import os

In [2]:
def print_edges(edges):
    print(','.join([str(e) for e in edges]))

# Считываем граф

Oставляем только большую компоненту связности

In [3]:
G = nx.DiGraph()

with open("assembly/K127/prelim_graph.gfa") as f:
    for line in f:
        line = line.split()
        line_type = line[0]
        
        # S 238024 ACCAATTAT KC:i:37210
        if line_type == "S":
            v_name = int(line[1])
            v_length = len(line[2])
            G.add_node(v_name, length=v_length)
        
        # L 238322 + 19590 - 55M
        if line_type == "L":
            v1 = int(line[1])
            v2 = int(line[3])
            G.add_edge(v1, v2)
            
print('Number of components:', nx.number_weakly_connected_components(G))      
            
# remain only largest component
new_G = nx.DiGraph()
for g in nx.weakly_connected_component_subgraphs(G):
    print(g.number_of_nodes())
    if new_G.number_of_nodes() < g.number_of_nodes():
        new_G = g.copy()
G = new_G.copy()

Number of components: 11
3532
1
1
1
1
1
1
1
1
1
1


# Табличка с референсами

Считываем файл ответа, как он есть

In [4]:
df_ref = pd.read_csv("refs/refs_edges.txt", header=None, names=["e"])

df_ref = df_ref["e"].str.split('\t', 1, expand=True)
df_ref.columns = ["e_id", "strains"]
df_ref = df_ref.set_index("e_id")
df_ref.index = df_ref.index.astype("int")
df_ref.head()

,strains
e_id,
4575684,SE11_1\tHS_1\tH7_1\tO104_1
4558554,H7_1\tH7_1\tH7_1\tH7_1\tO104_1\tO104_1
4412196,H7_1\tO104_1
4181032,H7_1
4491448,SE11_1\tHS_1\tO104_1


Оставляем только ребра из большой компоненты:

In [5]:
df_ref = df_ref.loc[list(G.nodes)]

Выводим рёбра, которые никому не принадлежат, на экран, и выкидываем их из таблицы:

In [6]:
nobody_edges = df_ref[df_ref["strains"].isnull()].index
print("{n} edges are Nobody".format(n=len(nobody_edges)), '\n')
print_edges(nobody_edges)

df_ref = df_ref.loc[set(df_ref.index) - set(nobody_edges)]

0 edges are Nobody 




Сплитим список референсов:

In [7]:
df_ref["strains"] = df_ref["strains"].str.split('\t')
df_ref["strains"] = df_ref["strains"].apply(lambda x: [s.rpartition('_')[0] for s in x])
df_ref["strains"] = df_ref["strains"].apply(Counter)
df_ref.head()

,strains
e_id,
4571136,"{'SE11': 1, 'HS': 1, 'H7': 1, 'O104': 1}"
4554754,"{'SE11': 1, 'HS': 1, 'O104': 1}"
4177934,{'H7': 1}
3997720,{'H7': 1}
4177948,{'HS': 1}


Считаем копийность каждого ребра:

In [8]:
df_ref["single_copy"] = df_ref["strains"].apply(lambda x: x.most_common(1)[0][1] == 1)
df_ref.head()

,strains,single_copy
e_id,,
4571136,"{'SE11': 1, 'HS': 1, 'H7': 1, 'O104': 1}",True
4554754,"{'SE11': 1, 'HS': 1, 'O104': 1}",True
4177934,{'H7': 1},True
3997720,{'H7': 1},True
4177948,{'HS': 1},True


Считываем длину рёбер:

In [9]:
df_length = pd.read_csv("edges_lengths.tsv", sep='\t', header=None, names=["length"])
df_length = df_length.astype("int")

df_ref = df_ref.join(df_length, how='inner')
df_ref.head()

,strains,single_copy,length
4571136,"{'SE11': 1, 'HS': 1, 'H7': 1, 'O104': 1}",True,7587
4554754,"{'SE11': 1, 'HS': 1, 'O104': 1}",True,1230
4177934,{'H7': 1},True,421
3997720,{'H7': 1},True,436
4177948,{'HS': 1},True,516


# Считываем профили

In [10]:
ref_profile = pd.read_csv("refs/profile.csv", header=None, index_col=0)
for i in range(1, 11):
    ref_profile[i] = ref_profile[i] / ref_profile[i].sum()
ref_profile

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
SE11,0.19,0.23,0.43,0.93,0.82,0.57,0.89,0.64,0.04,0.05
HS,0.06,0.70,0.03,0.05,0.08,0.34,0.04,0.03,0.09,0.53
H7,0.69,0.03,0.03,0.02,0.05,0.09,0.06,0.30,0.01,0.05
O104,0.06,0.04,0.51,0.00,0.05,0.00,0.01,0.03,0.86,0.37


In [11]:
desman_profile = pd.read_csv("desman_results/4_0/desman_freqs.csv", header=None, index_col=0, dtype=float)
desman_profile.index = desman_profile.index.astype(int)
desman_profile

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
0,0.695,0.035,0.031,0.001,0.031,0.092,0.062,0.303,0.001,0.031
1,0.056,0.715,0.030,0.033,0.067,0.332,0.033,0.031,0.091,0.539
2,0.187,0.215,0.437,0.964,0.867,0.574,0.904,0.635,0.035,0.033
3,0.062,0.035,0.501,0.001,0.034,0.002,0.001,0.031,0.873,0.396


Ищем соответствие между профилями:

In [12]:
ref_freqs = ref_profile.as_matrix()
ans_error = float("Inf")
ans_permut = None
for cur_permut in permutations(desman_profile.index):
    desman_freqs = desman_profile.loc[cur_permut, :].as_matrix()
    #print(cur_error, cur_permut)
    cur_error = ((ref_freqs - desman_freqs) ** 2).sum()
    if cur_error < ans_error:
        ans_error = cur_error
        ans_permut = cur_permut
print("Error:", ans_error)

Error: 0.007624


In [13]:
def invert_permutation(permutation):
    return [i for i, j in sorted(enumerate(permutation), key=lambda x: x[1])]

In [14]:
strains = list(ref_profile.iloc[invert_permutation(ans_permut), :].index.astype(str))
strains

['H7', 'HS', 'SE11', 'O104']

# Табличка ответов DESMAN

In [15]:
df_desman = pd.read_csv("desman_results/4_0/gene_assignment_etaS_df.csv", skiprows=1,
                  names=["e_id"] + strains)
df_desman['e_id'] = df_desman['e_id'].str[1:].astype("int")
df_desman = df_desman.set_index('e_id')
df_desman[strains] = df_desman[strains].astype('int')

#df_desman = df_desman.join(df_length, how='inner')

df_desman = df_desman.loc[set(df_ref.index) - set(nobody_edges)]
df_desman.head()

,H7,HS,SE11,O104
e_id,,,,
4571136,1,1,1,1
4554754,0,1,1,1
4177934,1,0,0,0
3997720,1,0,0,0
4177948,0,1,0,0


In [16]:
for cur_s in strains:
    df_ref[cur_s] = df_ref['strains'].apply(lambda x: int(cur_s in x))
    
df_ref.head()

,strains,single_copy,length,H7,HS,SE11,O104
4571136,"{'SE11': 1, 'HS': 1, 'H7': 1, 'O104': 1}",True,7587,1,1,1,1
4554754,"{'SE11': 1, 'HS': 1, 'O104': 1}",True,1230,0,1,1,1
4177934,{'H7': 1},True,421,1,0,0,0
3997720,{'H7': 1},True,436,1,0,0,0
4177948,{'HS': 1},True,516,0,1,0,0


# Точность DESMAN

In [17]:
df_ref.sort_index(inplace=True)
df_desman.sort_index(inplace=True)

In [18]:
right_answers = (df_ref[strains] == df_desman[strains]).sum(axis=1) == len(strains)
print("Accuracy on all edges: %.2f" % (right_answers.sum() / len(df_ref)))

Accuracy on all edges: 0.89


In [19]:
long_edges = df_ref['length'] > 500
print("Percent of long edges: %.2f" % (long_edges.sum() / len(df_ref)))
print("Accuracy on long edges: %.2f" % ((right_answers & long_edges).sum() / long_edges.sum()))

Percent of long edges: 0.60
Accuracy on long edges: 0.94


# Раскрашиваем граф для конкретного штамма

In [20]:
if not os.path.exists("bandage_colors"):
    os.makedirs("bandage_colors")


for cur_s in strains:
    print('\n\n', "_________________________", cur_s, '\n')
    
    selected_nodes = df_ref[df_ref[cur_s] == 1].index
    G_sub = G.subgraph(selected_nodes)
    print("components in reference subgraph:", nx.number_weakly_connected_components(G_sub))

    selected_nodes = df_desman[df_desman[cur_s] == 1].index
    G_sub = G.subgraph(selected_nodes)
    print("components in   DESMAN  subgraph:", nx.number_weakly_connected_components(G_sub))

    df_ref['color'] = "#b0b0b0"  # grey


    long = df_ref['length'] >= 500
    single = df_ref['single_copy']
    real_true = df_ref[cur_s] == 1
    desman_true = df_desman[cur_s] == 1

    df_ref.loc[~long & real_true, 'color'] = 'Brown'

    df_ref.loc[long & single  &  real_true  &  desman_true, 'color'] = 'Lime'
    df_ref.loc[long & ~single  &  real_true  &  desman_true, 'color'] = 'Green'

    df_ref.loc[long &  single  &  real_true  & ~desman_true, 'color'] = 'Teal'
    df_ref.loc[long & ~single  &  real_true  & ~desman_true, 'color'] = 'Navy'

    df_ref.loc[long &  single  & ~real_true  &  desman_true, 'color'] = 'Yellow'
    df_ref.loc[long & ~single  & ~real_true  &  desman_true, 'color'] = 'Orange'


    df_ref['strains_print'] = df_ref['strains'].apply(lambda x: ", ".join('{}({})'.format(k, v) for k, v in x.items()))
    df_ref['strains_print'] = df_ref['strains_print'].apply(lambda x: x.replace('(1)', ''))

    df_ref[['strains_print', 'color']].to_csv("bandage_colors/{}.csv".format(cur_s), index_label='name')
    
    print("\nlong FN")
    print_edges(df_ref[long & real_true & ~desman_true].index)
    
    print("\nlong FP")
    print_edges(df_ref[long & ~real_true & desman_true].index)



 _________________________ H7 

components in reference subgraph: 11
components in   DESMAN  subgraph: 87

long FN
4165848,4290510,4292958,4293044,4293292,4293724,4349592,4377090,4402772,4408410,4437740,4472206,4482858,4501256,4517984,4531232,4532488,4542978,4544570,4548504,4555250,4556342,4556628,4559216,4559712,4561686,4562796,4563126,4563656,4570420,4570746,4573688,4573856,4576560,4576816,4576902,4577406,4577464,4577730,4578218,4578376,4578470,4579176,4579420,4580740,4582838,4582864,4583478,4583722,4583788,4583818,4583958,4583988,4583990,4584084,4584130,4584136,4584154,4584202,4584216,4584218,4584254,4585394,4586124,4586212,4586392,4586590,4586668,4586712,4586750,4586764,4587000

long FP
4291784,4429784,4562000,4569642,4573706,4575648,4575704,4577470,4577618,4577940,4579930,4580136,4581922,4587874


 _________________________ HS 

components in reference subgraph: 1
components in   DESMAN  subgraph: 43

long FN
4487582,4586260,4586296,4586318,4586494,4586606,4586610,4586636,458691